## Sources

In [1]:
import pandas as pd
import numpy as np
import csv
import uuid

### SKOS Concept

In [2]:
PATH = "/home/STual/KG-cadastre/gentilly/data/SKOS_SOURCES.csv"

skos_srcs = pd.read_csv(PATH,header=0)
display(skos_srcs)

FileNotFoundError: [Errno 2] No such file or directory: '/home/STual/KG-cadastre/gentilly/data/SKOS_SOURCES.csv'

In [ ]:
# Print the list of column headers
print(skos_srcs.columns.tolist())

['skos:Concept', 'skos:inScheme', 'skos:broader', 'rdfs:label', 'skos:prefLabel@fr', 'skos:prefLabel@en', 'skos:altLabel@fr', 'skos:altLabel@en', 'skos:definition']


In [ ]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import SKOS, RDF, OWL

# Create a new RDF graph
g = Graph()

baseuri = Namespace("http://data.ign.fr/id/codes/cadastrenap/sourceType/")
g.bind('srctype', baseuri)

# Define the namespaces
cad = Namespace("http://data.ign.fr/def/cadastre#")
src = Namespace("http://data.ign.fr/def/sources#")
g.bind('cad', cad)
g.bind('src', src)

mlclasse= Namespace("http://data.ign.fr/id/codes/cadastrenap/mlClasse/")
g.bind('mlclasse', mlclasse)

# Iterate over each row in the DataFrame
for index, row in skos_srcs.iterrows():
    subject_uri = URIRef(baseuri + f"{str(row['skos:Concept'])}")
    g.add((subject_uri, RDF.type, OWL.NamedIndividual))
    g.add((subject_uri, RDF.type, cad.SourceType))
    g.add((subject_uri, RDF.type, SKOS.Concept))
    g.add((subject_uri, SKOS.inScheme, URIRef(cad + f"{str(row['skos:inScheme'])}")))
    g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@fr"],lang='fr')))

    if not pd.isnull(row['skos:altLabel@fr']):
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@fr"],lang='fr')))

    if not pd.isnull(row['skos:prefLabel@en']):
        g.add((subject_uri, SKOS.prefLabel, Literal(row["skos:prefLabel@en"],lang='en')))

    if not pd.isnull(row['skos:altLabel@en']):
        g.add((subject_uri, SKOS.altLabel, Literal(row["skos:altLabel@en"],lang='en')))

    if not pd.isnull(row['skos:broader']):
        g.add((subject_uri, SKOS.broader, URIRef(baseuri + f"{str(row['skos:broader'])}")))

    if not pd.isnull(row['skos:definition']):
        g.add((subject_uri, SKOS.definition, Literal(row["skos:definition"],lang='fr')))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastrenap/sourceType/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

srctype:CompteFoncier a cad:SourceType,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader srctype:ZoneDePage ;
    skos:definition "Ensemble de lignes et de cellules qui décrivent les parcelles d'un ou plusieurs propriétaires conjoints ou sucessifs"@fr ;
    skos:inScheme cad:SourceList ;
    skos:prefLabel "Zone située dans une page de matrice correspondant à un compte foncier"@fr .

srctype:EtatsDeSections_Ap_1822 a cad:SourceType,
        owl:NamedIndividual,
        skos:Concept ;
    skos:broader srctype:EtatsDeSections ;
    skos:definition "Registre d'états de sections créé après 1822."@fr ;
    skos:inScheme cad:SourceList ;
    skos:prefLabel "Etats de sections postérieurs à 1822"@fr .